In [2]:
import pandas as pd

df = pd.read_csv('train.tsv', sep='\t')

In [48]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(binary=True)
X = vectorizer.fit_transform(df["Phrase"][:31])

In [52]:
import torch

t = torch.from_numpy(X.todense())

#ajout du cas où on ne connait pas le mot
voca = vectorizer.get_feature_names() + list(['<unk>'])
unk = torch.zeros([t.shape[0],1], dtype=torch.int64)
t = torch.cat((t, unk), 1)

In [53]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):

        #x = [sent len, batch size]
        
        embedded = self.embedding(x)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

In [54]:
input_dim = len(voca)
embedding_dim = 100
hidden_dim = 250
output_dim = 1

model = RNN(input_dim, embedding_dim, hidden_dim, output_dim)